In [1]:
# ! pip install pydantic-ai

## Tools WITHOUT Context 

In [2]:
import random

from pydantic_ai import RunContext, Tool


def roll_dice(name: str = "Rolling dice") -> str:
    """Roll a six-sided dice and return the result."""
    return str(random.randint(1, 6))


rool_dice_tool = Tool(roll_dice)

In [ ]:
rool_dice_tool.function()

In [ ]:
print(rool_dice_tool.name)
print(rool_dice_tool.description)

In [ ]:
rool_dice_tool._parameters_json_schema

In [ ]:
from typing import Optional


def foobar(a: int, b: str, c: Optional[dict[str, list[float]]] = None) -> str:
    """Get me foobar.

    Args:
        a: apple pie
        b: banana cake
        c: carrot smoothie
    """
    return f"{a} {b} {c}"


foobar_tool = Tool(foobar)

print(foobar_tool.name)
print(foobar_tool.description)

In [ ]:
foobar_tool._parameters_json_schema

## Tools WITH Context (simple string)

In [8]:
def get_player_name(ctx: RunContext[str]) -> str:
    """Get the player's name."""
    return ctx.deps


get_player_name_tool = Tool(get_player_name, takes_ctx=True)

In [9]:
# ! pip install nest_asyncio

In [10]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
from pydantic_ai import Agent

agent_a = Agent(
    "openai:gpt-4o",
    deps_type=str,
    tools=[get_player_name_tool],
)

In [ ]:
result = agent_a.run_sync("Get players name", deps="Anne")
result

In [ ]:
result.all_messages()

## Tools WITH Context (pydantic)

In [14]:
from pydantic import BaseModel


class Player(BaseModel):
    name: str
    age: int


def get_player(ctx: RunContext[Player], additional_info: Optional[str] = None) -> str:
    """Get the player's name.

    Args:
        additional_info: Additional information which can be used.
    """
    return f"Name: {ctx.deps.name}, Age: {ctx.deps.age}, Additional info: {additional_info}"


get_player_tool = Tool(get_player, takes_ctx=True)

In [ ]:
print(get_player_tool.name)
print(get_player_tool.description)
print(get_player_tool._parameters_json_schema)

In [ ]:
agent_b = Agent(
    "openai:gpt-4o",
    deps_type=Player,
    tools=[get_player_tool],
)

player = Player(name="Luka", age=25)
result_b = agent_b.run_sync("Get player", deps=player)
result_b.all_messages()

## Dependency injection

In [17]:
# Initial Source: https://github.com/airtai/fastagency/blob/main/fastagency/api/dependency_injection.py

from functools import wraps
from inspect import signature
from typing import Any, Callable


def inject_params(f: Callable[..., Any], ctx: Any) -> Callable[..., Any]:

    @wraps(f)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        kwargs.pop("ctx", None)
        return f(**kwargs, ctx=ctx)

    sig = signature(f)
    new_params = [param for name, param in sig.parameters.items() if name != "ctx"]

    wrapper.__signature__ = sig.replace(parameters=new_params)  # type: ignore[attr-defined]

    return wrapper

In [18]:
ctx = RunContext(
    deps=Player(name="Luka", age=25),
    retry=get_player_tool.max_retries,  # Not sure
    messages=[],  # Not sure
    tool_name=get_player_tool.name,
)
f_injected = inject_params(f=get_player_tool.function, ctx=ctx)

In [ ]:
f_injected.__signature__

In [ ]:
f_injected(additional_info="Hello")

## AG2 Integration

In [21]:
import os

from autogen import AssistantAgent, UserProxyAgent

In [22]:
config_list = [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
)

chatbot = AssistantAgent(
    name="chatbot",
    llm_config={"config_list": config_list},
)

In [28]:
from typing import Dict


def create_function_schema(name: str, description: str, parameters: Dict[str, Any]) -> Dict[str, Any]:
    # parameters are:
    #     {'properties': {'additional_info': {'anyOf': [{'type': 'string'},
    #     {'type': 'null'}],
    #    'description': 'Additional information which can be used.',
    #    'title': 'Additional Info'}},
    #  'required': ['additional_info'],
    #  'type': 'object',
    #  'additionalProperties': False}

    # response should be:
    # {'type': 'function', 'function': {'description': "Get the player's name.", 'name': 'get_player', 'parameters': {'type': 'object', 'properties': {'additional_info': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'additional_info'}}, 'required': []}}
    return {
        "type": "function",
        "function": {
            "name": name,
            "description": description,
            "parameters": parameters,
        },
    }

In [ ]:
get_player_tool._parameters_json_schema

In [ ]:
function_schema = create_function_schema(
    name=get_player_tool.name,
    description=get_player_tool.description,
    parameters=get_player_tool._parameters_json_schema,
)
function_schema

In [33]:
tool = get_player_tool

f_injected_ctx = inject_params(f=tool.function, ctx=ctx)

In [ ]:
user_proxy.register_for_execution(name=tool.name)(f_injected_ctx)
# chatbot.register_for_llm(description=tool.description, name=tool.name)(f_injected_ctx)
chatbot.update_tool_signature(function_schema, is_remove=False)

In [ ]:
chatbot.llm_config["tools"]

In [ ]:
user_proxy.initiate_chat(
    recipient=chatbot, message="Get player, for additional information use 'goal keeper'", max_turns=3
)